## 第一階段：取得所有必要的職缺網址

In [167]:
from datetime import datetime
import json
import random
import re
import time
from urllib.parse import unquote, quote

from bs4 import BeautifulSoup
import requests

In [140]:
search_url = input('請輸入搜尋頁面網址 ')

請輸入搜尋頁面網址 https://www.104.com.tw/jobs/search/?ro=0&keyword=%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB&area=6001002000%2C6001001000&jobsource=2018indexpoc


In [185]:
kw = input('請輸入關鍵字 ')
search_url = \
f'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword={quote(kw)}&area=\
6001002000%2C6001001000&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc'

請輸入關鍵字 Python


In [3]:
def url_to_soup(url):
    head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4'} 
    resp = requests.get(url, headers=head)
    soup = BeautifulSoup(resp.text)
    return soup

In [4]:
def get_page_urls(search_url):
    soup = url_to_soup(search_url)
    jobs = soup.select('.js-job-link')
    page_urls = []
    for job in jobs:
        try:
            page_urls.append('https:' + job['href'].split('&')[0])
        except:
            page_urls.append('https:' + job['href'])
    return page_urls

In [186]:
def get_all_urls(search_url):
    job_urls = []
    for page in range(1, 151):
        if 'page=' in search_url:
#           用 f-string 就不必使用 str(page)了，因為page必轉為str
            search_url = re.sub(r'page=\d*', f'page={page}',
                                search_url)
        else:
            search_url = search_url + f'&page={page}'
            
        print(f'{page}頁', end='\t')
        page_urls = get_page_urls(search_url)
        job_urls.extend(page_urls)
        
        if len(page_urls) < 20:
            break
        time.sleep(random.uniform(4, 10))

    print('')
    print('總共收集', len(job_urls), '個職缺網址')
    print('完成')
    return job_urls

In [187]:
result = get_all_urls(search_url)

1頁	2頁	3頁	4頁	5頁	6頁	7頁	8頁	9頁	10頁	11頁	12頁	13頁	14頁	15頁	16頁	17頁	18頁	19頁	20頁	21頁	22頁	23頁	24頁	25頁	26頁	27頁	28頁	29頁	30頁	31頁	32頁	33頁	34頁	35頁	36頁	37頁	38頁	39頁	40頁	41頁	42頁	43頁	44頁	45頁	46頁	47頁	48頁	49頁	50頁	51頁	52頁	53頁	54頁	55頁	56頁	57頁	58頁	59頁	60頁	61頁	62頁	63頁	64頁	65頁	66頁	67頁	68頁	69頁	70頁	71頁	72頁	73頁	74頁	75頁	76頁	77頁	78頁	79頁	80頁	81頁	82頁	83頁	84頁	85頁	86頁	87頁	88頁	89頁	90頁	91頁	92頁	93頁	94頁	95頁	96頁	97頁	98頁	99頁	100頁	101頁	102頁	103頁	104頁	105頁	106頁	107頁	108頁	109頁	110頁	111頁	112頁	113頁	114頁	
總共收集 2283 個職缺網址
完成


### 將結果寫入JSON檔

#### 先從網址取出關鍵字

In [188]:
keyword_raw = re.search(r'keyword=[^&]*', search_url).group().split('=')[1]
keyword = unquote(keyword_raw)
keyword

'Python'

#### 記錄當下時間，以指定格式

In [189]:
timestamp = datetime.now().strftime('%m%d-%H%M')
timestamp

'0428-0245'

In [190]:
len(result)

2283

In [191]:
result_set = set(result)

In [192]:
len(result_set)

2260

In [193]:
result_set.discard('https:javascript:void(0);')
result_tuple = tuple(result_set)
len(result_tuple)

2259

In [194]:
with open(f'job_urls/{keyword}-{timestamp}.json', 'w') as f:
    json.dump(result_tuple, f)

In [195]:
# filename = input()
# with open(f'data/{filename}.json', encoding='utf8') as file:
#     one = json.load(file)

In [196]:
block = ('前端', '測試', '分析', 'front', 'php', 'java', 'asp', 'c#', 'ios', 
        'android', 'react', 'vue', 'angular', 'ux', 'ui', 'rail',
        'ruby', '.net', 'core', 'test', '視覺', '資安', '網頁設計', 'html',
        'css', 'jquery', '影像', '行政', 'manager', '韌體', 'c+', 'unity',
        '硬體', '介面', 'qa', 'quality', '維護', '研發替代役', 'designer',
        '美術', '物聯網', 'iot', '介面設計', '嵌入', 'devops', 'scientist',
        '人工智', '圖像', '算法', 'hardware', '硬體', '品質', 'lte', '實習',
        '科學家', 'firmware', 'sre', 'reliability', '可靠性', 'embed', '顧問',
        'vr', '助理', '自動駕', '深度學習', 'nlp', '工讀', 'intern', '兼職',
        '襄理', '副理', '經理', 'fae', 'bot', 'crm', '主管', '主任', 'analyst',
        '網管', 'integr', 'pm', '3d', '動畫', '行銷', '研究員', '機率', '特效',
        'mis', '光學', '約聘', '總監', 'algorithm','檢測', '支援', '客服', '製作人',
        'erp', '網站設計', '編輯', '專員', '總監', '小編', '機器人', '企劃', 'node',
        '補習班', '證券', '老師', '自動化', '機器學習', '講師', 'learning', '教師',
        '交易員', '驗證', '汽車', '財務工程')

# app 遊戲 bi

In [197]:
filename = input('請輸入檔名：')
with open(f'job_urls/{filename}.json', encoding='utf8') as file:
    urls = json.load(file)
len(urls)

請輸入檔名：後端工程師-0425-2321


1503

In [198]:
def urls_to_data(result, block, block_urls):
    print(datetime.now().strftime('%H:%M:%S'))
    print(f'開始處理，總共有 {len(result)} 個項目')
    block_list = block
    data = []
    item = 1
    number = 1
    for url in result:
        blocked = False
        job = {}
        
        if url in block_urls:
            print('◆', end='\t')
            number = number + 1
            continue
        
        try:
            soup = url_to_soup(url)
            job['資料編號'] = f'第 {item} 項'
            job['職位網址'] = f' {url} '            
            job['職位名稱'] = soup.find_all('meta')[4]['content'].rstrip(
                ' - 104人力銀行')
            
            if 'python' in job['職位名稱'].lower():
                pass
            else:
#                 blocked = any(( item for item in block_list 
#                                if item in job['職位名稱'].lower()))
                for term in block_list:
                    if term in job['職位名稱'].lower():
                        blocked = True
                        break
            if blocked:
                print('×', end='\t')
                number = number + 1
                time.sleep(random.uniform(3, 9))
                continue

            job['工作地點'] = soup.find('dd', 'addr').text.strip().split()[0]
            job['工作內容'] = soup.find('p').text.replace('\r', '').replace(
                '\t', '').replace('\u3000', '').replace('\xa0', '')
            job['工作內容'] = job['工作內容'][:300]
            
            job['擅長工具'] = soup.select('.info')[1].find_all('dd')[-3].text
            job['工作技能'] = soup.select('.info')[1].find_all('dd')[-2].text
            job['其它條件'] = soup.select('.info')[1].find_all(
                'dd')[-1].text.replace('\r', '').replace('\t', '').replace(
                '\u3000', '').replace('\xa0', '')
            job['其它條件'] = job['其它條件'][:300]
            
        except:
            print('×', end='\t')
            number = number + 1
            time.sleep(random.uniform(3, 9))
            continue

        for k, v in job.items():
            if v in ('不拘', '未填寫'):
                job[k] = ''
        data.append(job)
        print(number, end='\t')
        item = item + 1
        number = number + 1
        time.sleep(random.uniform(3, 9))
    print('')
    print(f'完成，實際抓取 {len(data)} 個項目')
    print(datetime.now().strftime('%H:%M:%S'))
    return data

In [199]:
# data = urls_to_data(one, block=block)

In [200]:
data = urls_to_data(result_tuple, block=block, block_urls=urls)

02:46:36
開始處理，總共有 2259 個項目
×	×	◆	×	5	6	7	×	×	10	11	◆	13	◆	×	×	×	×	×	20	×	×	×	24	×	×	27	×	×	30	×	32	×	×	◆	◆	×	×	×	×	×	×	43	44	◆	◆	×	×	×	×	×	×	53	×	×	56	×	×	◆	×	×	◆	63	×	65	×	67	68	×	×	×	×	×	×	×	×	×	×	×	80	×	82	83	×	×	×	◆	◆	89	×	◆	×	×	94	×	×	×	×	◆	◆	×	×	103	×	×	×	◆	108	◆	×	×	×	×	×	115	116	117	118	◆	×	×	◆	123	◆	×	×	×	×	×	×	×	×	×	134	×	136	×	×	139	140	×	×	◆	144	◆	×	×	148	149	150	×	×	×	154	155	156	×	×	×	×	×	◆	×	164	165	×	◆	168	×	170	◆	×	173	×	◆	◆	×	×	×	×	×	◆	×	×	185	×	187	×	×	190	×	×	193	×	×	196	197	×	×	200	×	202	203	×	×	×	207	×	×	◆	211	×	×	×	×	216	×	218	×	220	221	×	×	◆	×	×	227	×	×	×	×	×	×	×	×	×	×	238	239	240	×	×	×	×	245	◆	247	×	×	◆	×	×	×	◆	255	256	257	258	259	260	×	262	×	◆	×	◆	×	×	269	◆	×	×	273	◆	×	×	×	×	×	×	×	×	◆	×	◆	×	×	×	289	290	×	×	293	294	◆	◆	◆	298	×	×	×	×	303	×	305	×	×	×	◆	310	×	312	×	◆	×	316	×	×	×	320	×	×	×	324	◆	◆	327	328	×	330	331	×	333	◆	×	336	337	338	339	×	◆	×	×	×	×	×	◆	348	×	350	×	×	×	◆	×	×	×	×	359	◆	×	×	×	×	◆	366	367	×	×	370	371	×	×	×	×	×	377	×	×	×	×	382	×	×	385	×	387	×	389	×	

In [202]:
data

[{'資料編號': '第 1 項',
  '職位網址': ' https://www.104.com.tw/job/?jobno=6gjv6 ',
  '職位名稱': '運維工程師(無需輪班/無經驗/應接畢業生可) | 鑫利數位科技有限公司 | 台北市內湖區',
  '工作地點': '台北市內湖區台北市內湖區瑞光路513巷26號2F(奔騰大樓)',
  '工作內容': '★想進入遊戲產業但又限於無相關經驗遭拒門外嗎？這裡正有個機會～★你會獲得：遊戲產業入門門票／跨國團隊合作機會／無限發展空間★若你具備以下條件，熱烈歡迎加入我們 1. 具有Linux & Windows 基礎知識並能協助完成日常運維工作2. TCP/IP & Domain 基礎知識3.了解zabbix 等監控軟體使用4. 熟悉haproxy,nginx,uwsgi,redis,mysql,mongodb5.負責團隊工程效率的提升，維護改善 CI/CD 工具與流程的支撐。  [大加分條件]1.使用源代碼控制工具（Git，SVN等）在團隊協作環境中的經驗2.有建立安全網絡或在網絡中實施安全措施的經驗',
  '擅長工具': 'Linux、Windows 2000、Windows XP、Windows 8、Python、MySQL、Vmware、AWS',
  '工作技能': '網路系統配置、網路應用軟體操作',
  '其它條件': '此工作須須配合On call'},
 {'資料編號': '第 2 項',
  '職位網址': ' https://www.104.com.tw/job/?jobno=5wizg ',
  '職位名稱': '網路工程師-Akamai,FireEye,F5產品(歡迎1-3年經驗) | 零壹科技股份有限公司_Zero One Tech | 台北市內湖區',
  '工作地點': '台北市內湖區內湖路一段360巷8號10樓',
  '工作內容': '1.售前支援:架構討論,PoC測試及文件提供2.售後支援:安裝設定,維護,處理故障及礙排除,簡易教育訓練3.產品測試及整合Demo',
  '擅長工具': 'Perl、Python、TCL',
  '工作技能': '',
  '其它條件': '1.具備網路架構,TCP/IP,資訊安全等經驗.2.熟悉Windows/Linux作業系統,Mai

In [201]:
timestamp=datetime.now().strftime('%m%d-%H%M')

with open(f'data/{keyword}-{timestamp}.json', 'w') as f:
    json.dump(data, f)

In [36]:
from collections import Counter

In [87]:
Counter(i['工作地點'][:6] for i in data).most_common()

[('台北市內湖區', 170),
 ('台北市信義區', 111),
 ('台北市大安區', 97),
 ('台北市中山區', 75),
 ('台北市南港區', 65),
 ('台北市松山區', 57),
 ('新北市中和區', 44),
 ('台北市中正區', 43),
 ('新北市新店區', 42),
 ('新北市汐止區', 36),
 ('台北市北投區', 19),
 ('新北市土城區', 19),
 ('新北市板橋區', 19),
 ('新北市三重區', 11),
 ('台北市士林區', 11),
 ('新北市五股區', 9),
 ('台北市大同區', 6),
 ('新北市深坑區', 6),
 ('新北市新莊區', 5),
 ('新北市樹林區', 4),
 ('新北市林口區', 4),
 ('台北市萬華區', 4),
 ('新北市永和區', 3),
 ('台北市文山區', 3),
 ('新北市泰山區', 2),
 ('新北市蘆洲區', 1),
 ('新北市淡水區', 1),
 ('新北市八里區', 1),
 ('新北市三峽區', 1),
 ('新北市瑞芳區', 1)]

In [21]:
import json

In [24]:
json.dumps(aa)

'[1, 2, 3]'

In [25]:
with open('data.json', 'w') as f:
    json.dump(aa, f)